In [1]:
import dask
from helpers.loading import *
from helpers import config


In [2]:
all_data_bbo = {}
for market in config["transatlantic"]["markets"]["list"]:
    all_data_bbo[market] = load_all_data_dask(market, signal="bbo", precision="D",cols=["date","price","bid-price","ask-price"])

all_data_trade = {}
for market in config["transatlantic"]["markets"]["list"]:
    all_data_trade[market] = load_all_data_dask(market,precision="ms")

In [36]:
all_trades = dd.concat(list(all_data_trade.values()))
all_trades.head()

,date,price
0,2005-07-15 06:54:05.498000+00:00,1848.0
1,2005-07-15 06:58:22.561000+00:00,1847.0
2,2005-07-15 06:58:28.643000+00:00,1840.0
3,2005-07-15 07:02:04.928000+00:00,1840.0
4,2005-07-15 07:02:05.409000+00:00,1840.0


### Daily number of trades

In [37]:
daily_trades = all_trades.copy()
daily_trades["date"] = daily_trades["date"].round("D")
n_daily_trade_avg = daily_trades.groupby("date").count().mean().compute()

In [38]:
print(f" there was {n_daily_trade_avg.price:0.1f} trades per day on average")

 there was 16277.0 trades per day on average


### Average period between trades 

In [56]:
all_periods = all_trades.reset_index().date.diff(1).dropna()
all_periods < all_periods[pd.Timedelta(1, "h")]
mean_period = all_periods.mean().compute()

In [60]:
print(f"there are {mean_period.total_seconds():0.2f}s on average between two trades")

there are 6.26s on average between two trades


### Average bid-ask spread 

In [4]:
bbo_df = dd.concat(list(all_data_bbo.values()))
bbo_df.head()

,date,bid-price,ask-price,price
4,2005-07-15 00:00:00+00:00,1830.0,1850.0,1840.0
5,2005-07-15 00:00:00+00:00,1830.0,1848.0,1839.0
6,2005-07-15 00:00:00+00:00,1850.0,1848.0,1849.0
7,2005-07-15 00:00:00+00:00,1830.0,1848.0,1839.0
8,2005-07-15 00:00:00+00:00,1832.0,1848.0,1840.0


In [6]:
bbo_df["spread"] = bbo_df["bid-price"]-bbo_df["ask-price"]
mean_spread = bbo_df.spread.mean().compute()

In [ ]:
print(f)